In [ ]:
from spaces import *
import graph_utils
%reload_ext autoreload
%autoreload 2
                
space = GridSpace([100,100])
space.add_rect(Rect(0., 0., 1.1, 0.1))
space.add_rect(Rect(0.1, 0.18, 0.3, 0.15))
space.add_rect(Rect(0.45, 0.41, 0.1, 0.4))
space.add_rect(Rect(0.7, 0.18, 0.2, 0.15))
space.checkpoint_vis()

theta1_start, theta2_start = 0.1, 0.2
theta1_end, theta2_end = 0.35, 0.3
l1, l2 = 0.2, 0.2
arm_start = Arm(theta1_start, theta2_start, l1, l2)
arm_end = Arm(theta1_end, theta2_end, l1, l2)
arm = Arm(0., 0., l1, l2)

c_space = create_arm_c_space(space, arm, dims=space.dims)
c_space.checkpoint_vis()

space.add_arm(arm_start, mark=None, mark_vis=BLUE)
space.add_arm(arm_end, mark=None, mark_vis=LIGHT_BLUE)

searcher = graph_utils.SingleQuery()

In [ ]:
c_space.reset_vis()
path = searcher.search_path(c_space, (theta1_start, theta2_start), 
                           (theta1_end, theta2_end), animate=False)
c_space.draw_point([theta1_start, theta2_start], color=BLUE)
c_space.draw_point([theta1_end, theta2_end], color=LIGHT_BLUE)
_=c_space.display('c_space.png')

In [ ]:
_=space.display('space.png')      

In [ ]:
space.animate_arm(arm_start, path, video_file='path.avi')